In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot

In [ ]:
# downloaded from /pnfs/uboone/persistent/users/markross/MiniBooNEDatasets2023/
f = uproot.open("data_files/output_osc_mc_detail_1.root")

print(f["MiniBooNE_CCQE"].keys())


In [ ]:
# from data_files/CombinedFunctions_from_Fortran/CombinedTypes.h
particle_types = {
    1: "gamma",
    2: "positron",
    3: "electron",
    4: "neutrino",
    5: "mu+",
    6: "mu-",
    7: "pi0",
    8: "pi+",
    9: "pi-",
    10: "K0L",
    11: "K+",
    12: "K-",
    13: "neutron",
    14: "proton",
    15: "anti-proton",
    16: "K0S",
    17: "eta",
    18: "lambda",
    19: "sigma+",
    20: "sigma0",
    21: "sigma-",
    22: "xi0",
    23: "xi-",
    24: "omega-",
    25: "anti-neutron",
    26: "anti-lambda",
    27: "anti-sigma-",
    28: "anti-sigma0",
    29: "anti-sigma+",
    30: "anti-xi0",
    31: "anti-xi+",
    32: "anti-omega+",
    45: "deuteron",
    46: "triton",
    47: "alpha",
    48: "geantino",
    49: "He3",
    50: "Cerenkov"
}


In [ ]:
df = f["MiniBooNE_CCQE"].arrays(["EventNumber", "Weight", "PassOsc", "RecoEnuQE", "Energy", "CosTheta", "TrueEnergy", "NuType", "NFSP", "FSPType", "MomX", "MomY", "MomZ", "MomT"], library="pd")

# from data_files/SomeMiniBooNEDetails_on_files_v2.pdf
nu_types = {
    1: "numu",
    2: "numubar",
    3: "nue",
    4: "nuebar"
}
df["nu_type"] = df["NuType"].apply(lambda x: nu_types[x])
df.drop(columns=["NuType"], inplace=True)

df